# Import Data

### Categories

To label the files, the file names will be tokenized, and if a file_name token matches with a category token, the intersection will result in a +1 score.

This isnt the best approach, because a name with "due" isn't always "due dilligence".

But this is better than manually labeling 500,000 files


In [2]:
import pandas as pd

In [3]:
df_cat = pd.read_excel("SPCategories.xlsx")

In [4]:
df_cat  = df_cat.drop(['◢','System.Collections.Generic.List<string>',"spClass.ListCategory"],axis=1)
df_cat  = df_cat.rename(columns={"spClass.ListCategory": "cat_num", "Count = 52": "cat_name"})
df_cat["cat_name"] = df_cat["cat_name"].str.strip('"')
df_cat = df_cat.iloc[:-1]
df_cat = df_cat.drop(index=0).reset_index(drop=True)
df_cat.head(5) 

,cat_name
0,Article
1,Audited Financial Statements
2,AUM
3,"Capital: committed, invested"
4,CCO Notes


### Now the file_names dataset

In [5]:
df_names = pd.read_csv('data.csv', usecols=[0])
df_names = df_names.fillna("")
#df_names = df_names[df_names["Name"].apply(lambda x: len(x) >= 6)]
df_names

,Name
0,PMC Q3 2024.pptx
1,Quarterly Manager Blurb - 2024.xlsx
2,GTF Exposure Snapshot - Sep 2024.xlsx
3,Janus Henderson Biotechnology Innovation Fund ...
4,2024-11-07-Documents By Last Modified Date.xlsx
...,...
588217,sharedStrings.xml
588218,styles.xml
588219,vbaProject.bin
588220,.rels


# Set file categories as dataframe columns

In [6]:
import numpy as np

df_result = pd.DataFrame(np.nan, index=df_names.index, columns=df_cat.iloc[:, 0].values)
df_result.insert(0, df_names.columns[0], df_names)

# Preprocess category names

In [7]:
import string
import re

# List of stop words to remove
stop_words = ["or", "and", "the", "of", "to", "in", "for", "on", "at", "a", "is", "it", "by", "an"]

# Convert column names to lowercase and remove stop words
stop_words_pattern = r'\b(?:' + '|'.join(map(re.escape, stop_words)) + r')\b'

df_result.columns = (
    df_result.columns.str.lower()
    .str.replace(f"[{string.punctuation}]", "", regex=True)
    .str.replace(stop_words_pattern, '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)  # Replace any spaces with underscores
    .str.strip('_')  # Remove leading/trailing underscores
)

df_result.head(3)

,name,article,audited financial statements,aum,capital committed invested,cco notes,compliance other,compliance business continuity manual,conference call,conference meeting notes,...,reference notes,risk management policy,risk report,service provider information,side letter,tax information,term sheet,unaudited financial statements,valuation policy,wire instructions
0,PMC Q3 2024.pptx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Quarterly Manager Blurb - 2024.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GTF Exposure Snapshot - Sep 2024.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Tokenize file names
File extentions are removed

In [8]:
import re
from transformers import BertTokenizer
import pandas as pd
from nltk.corpus import stopwords

# Ensure that the stopwords list is downloaded
import nltk
nltk.download('stopwords')

# Define the list of stop words
stop_words = set(stopwords.words('english'))

# Load a pretrained tokenizer from Hugging Face (e.g., BERT)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to remove file extension and tokenize names, remove stop words, and remove tokens with length <= 2
def tokenize_name_huggingface(name):
    # Remove file extension (anything after the last dot)
    name_without_extension = re.sub(r'\.[^.]*$', '', name)
    
    # Use the Hugging Face tokenizer to tokenize
    encoded = tokenizer(name_without_extension, truncation=True, padding=True, max_length=128)
    
    # Extract tokens (you can modify this to only use input_ids or other fields as needed)
    tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'])
    
    # Remove any special tokens (like [CLS], [SEP]) and stop words
    tokens = [token for token in tokens if token not in tokenizer.all_special_tokens]
    
    # Further filter out short tokens and stop words
    return [token.lower() for token in tokens if len(token) > 2 and token.lower() not in stop_words]

# Apply the Hugging Face tokenizer to the 'name' column
df_result['name_tokens'] = df_result['name'].apply(tokenize_name_huggingface)
df_result

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gregory.fatouras\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,name,article,audited financial statements,aum,capital committed invested,cco notes,compliance other,compliance business continuity manual,conference call,conference meeting notes,...,risk management policy,risk report,service provider information,side letter,tax information,term sheet,unaudited financial statements,valuation policy,wire instructions,name_tokens
0,PMC Q3 2024.pptx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[##c, ##3, 202, ##4]"
1,Quarterly Manager Blurb - 2024.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[quarterly, manager, blur, ##b, 202, ##4]"
2,GTF Exposure Snapshot - Sep 2024.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[##f, exposure, snaps, ##hot, sep, 202, ##4]"
3,Janus Henderson Biotechnology Innovation Fund ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[jan, ##us, henderson, biotechnology, innovati..."
4,2024-11-07-Documents By Last Modified Date.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[202, ##4, documents, last, modified, date]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588217,sharedStrings.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[shared, ##st, ##ring, ##s]"
588218,styles.xml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[styles]
588219,vbaProject.bin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[##ba, ##pro, ##ject]"
588220,.rels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


# Calculate scores between tokenized file_name lists, and category columns

In [21]:
df_result.to_csv('tokenized.csv', index=False)

In [9]:
import pandas as pd
import random
from tqdm import tqdm

# Assuming df_result already contains 'name_tokens' and the other columns

# Get the column names (excluding 'name' and 'name_tokens')
column_names = [col for col in df_result.columns if col not in ['name', 'name_tokens']]

# Function to calculate matches
def score_row(tokens, column_names):
    scores = {col: 0 for col in column_names}  # Initialize scores
    for token in tokens:
        for col in column_names:
            # Case-insensitive match and count matching tokens
            if token.lower() in col.lower():
                scores[col] += 1
    return scores

# Create empty lists to store the results
result_list = []
no_match_list = []

# Apply the scoring row by row with tqdm for progress tracking
for idx, row in tqdm(df_result.iterrows(), total=len(df_result), desc="Processing rows"):
    tokens = row['name_tokens']  # List of tokens from the 'name_tokens' column
    scores = score_row(tokens, column_names)  # Calculate match scores
    
    # Find the highest value column
    highest_value_column = max(scores, key=scores.get)
    highest_value = scores[highest_value_column]

    # If there is a match (highest value > 0), add to match_df
    if highest_value > 0:
        result_list.append({
            'name': row['name'],
            'token': row['name_tokens'],
            'highest_value_column': highest_value_column,
            'highest_value': highest_value
        })
    else:
        # Add to no_match_df if no match (highest value == 0)
        no_match_list.append({
            'name': row['name'],
            'token': row['name_tokens'],
            'highest_value_column': None,  # No match found
            'highest_value': 0  # No match score
        })

# Create DataFrame from the matched results
match_df = pd.DataFrame(result_list)

# Create DataFrame from the no-matched results
no_match_df = pd.DataFrame(no_match_list)

# Handle ties in match_df: If there is a tie, randomly pick one
if not match_df.empty:
    match_df['highest_value_column'] = match_df.groupby('name')['highest_value_column'].transform(
        lambda x: x.sample(1).iloc[0] if len(x) > 1 else x.iloc[0])

# Display the match_df and no_match_df for verification
print("Matched Results (match_df):")
print(match_df)

print("\nNo-Match Results (no_match_df):")
print(no_match_df)

# Optionally, save the results as CSV files
match_df.to_csv('match_results.csv', index=False)
no_match_df.to_csv('no_match_results.csv', index=False)


Processing rows: 100%|███████████████████████████████████████████████████████| 588222/588222 [00:58<00:00, 9973.67it/s]


Matched Results (match_df):
                                                     name  \
0                     Quarterly Manager Blurb - 2024.xlsx   
1         2024-11-07-Documents By Last Modified Date.xlsx   
2       Verition Commentary & Risk Report_September 20...   
3       Saba Capital Carry Neutral Tail Fund - Risk Re...   
4       Open_Protocol_One River Asset Management, LLC_...   
...                                                   ...   
204785                                          parser.js   
204786                                      dom-parser.js   
204787                         Emergency Contact List.doc   
204788                                      calcChain.xml   
204789                                      calcChain.xml   

                                                    token  \
0               [quarterly, manager, blur, ##b, 202, ##4]   
1             [202, ##4, documents, last, modified, date]   
2       [##rit, ##ion, commentary, risk, report, septe..

In [10]:
match_df.head(5)

,name,token,highest_value_column,highest_value
0,Quarterly Manager Blurb - 2024.xlsx,"[quarterly, manager, blur, ##b, 202, ##4]",monthly quarterly annual update,1
1,2024-11-07-Documents By Last Modified Date.xlsx,"[202, ##4, documents, last, modified, date]",monthly quarterly annual update,1
2,Verition Commentary & Risk Report_September 20...,"[##rit, ##ion, commentary, risk, report, septe...",risk report,2
3,Saba Capital Carry Neutral Tail Fund - Risk Re...,"[##ba, capital, carry, neutral, tail, fund, ri...",risk report,2
4,"Open_Protocol_One River Asset Management, LLC_...","[open, protocol, one, river, asset, management...",onepager,1


# Train Model

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming match_df is your dataframe
# Convert the list of tokens in 'token' column to a single string for vectorization
match_df['token_str'] = match_df['token'].apply(lambda x: ' '.join(x))

# Define the features (X) and the target (y)
X = match_df['token_str']  # Tokens as a single string
y = match_df['highest_value_column']  # The column to predict

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorizing the name tokens using TF-IDF
vectorizer = TfidfVectorizer(max_features=50000)  # Limit the number of features for efficiency
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Initialize and train the Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

# Predict the labels for the test set
y_pred = model.predict(X_test_vec)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Classification report for more detailed performance evaluation
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9911
Classification Report:


C:\Users\gregory.fatouras\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                                        precision    recall  f1-score   support

                               article       0.99      0.96      0.97       248
          audited financial statements       1.00      0.99      1.00      4130
            capital committed invested       0.99      1.00      0.99      4534
                             cco notes       0.98      0.98      0.98       733
 compliance business continuity manual       0.99      0.97      0.98       529
                      compliance other       0.99      0.94      0.96       115
                       conference call       1.00      0.98      0.99       879
              conference meeting notes       1.00      0.98      0.99       603
                         current event       1.00      0.95      0.97       212
                         due diligence       0.99      0.98      0.98       349
                  employee information       0.99      0.99      0.99      1659
                            fact sheet 

C:\Users\gregory.fatouras\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\gregory.fatouras\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
no_match_df

,name,token,highest_value_column,highest_value
0,PMC Q3 2024.pptx,"[##c, ##3, 202, ##4]",None,0
1,GTF Exposure Snapshot - Sep 2024.xlsx,"[##f, exposure, snaps, ##hot, sep, 202, ##4]",None,0
2,Janus Henderson Biotechnology Innovation Fund ...,"[jan, ##us, henderson, biotechnology, innovati...",None,0
3,09_30_24_Abbey_Futures_Fund_NAVs.xlsx,"[abbey, futures, fund, ##vs]",None,0
4,GF Q3 2024.pdf,"[##f, ##3, 202, ##4]",None,0
...,...,...,...,...
383427,sharedStrings.xml,"[shared, ##st, ##ring, ##s]",None,0
383428,styles.xml,[styles],None,0
383429,vbaProject.bin,"[##ba, ##pro, ##ject]",None,0
383430,.rels,[],None,0


In [13]:
# Your code for creating the 'predicted_category' column
no_match_df['token_str'] = no_match_df['token'].apply(lambda x: ' '.join(x))

# Vectorize the token strings using the already fitted TfidfVectorizer
X_no_match_vec = vectorizer.transform(no_match_df['token_str'])

# Use the trained model to predict the categories for no_match_df
no_match_df['predicted_category'] = model.predict(X_no_match_vec)

# Export only the 'name' and 'predicted_category' columns to an Excel file
no_match_df[['name', 'predicted_category']].to_excel('predicted_classes.xlsx', index=False)

In [18]:
import os
import matplotlib.pyplot as plt

# Ensure the images folder exists
output_folder = "./images"
os.makedirs(output_folder, exist_ok=True)

# Plot token importance for each unique predicted category
for predicted_category in no_match_df['predicted_category'].unique():
    print(f"Plotting for predicted category: {predicted_category}")

    # Filter rows for this predicted category
    subset_df = no_match_df[no_match_df['predicted_category'] == predicted_category]
    
    # Aggregate token importance across all rows for this category
    token_importance = {}
    category_index = list(class_labels).index(predicted_category)
    
    for _, row in subset_df.iterrows():
        tokens = row['token_str'].split()
        token_indices = [feature_names.tolist().index(token) for token in tokens if token in feature_names]
        for token, idx in zip(tokens, token_indices):
            token_importance[token] = token_importance.get(token, 0) + coefficients[category_index][idx]

    # Sort tokens by aggregate importance
    sorted_tokens = sorted(token_importance.items(), key=lambda x: x[1], reverse=True)
    if not sorted_tokens:
        print(f"No tokens found for category '{predicted_category}'")
        continue

    top_tokens, top_coefficients = zip(*sorted_tokens[:10])  # Top 10 tokens

    # Plot
    plt.figure(figsize=(10, 6))
    plt.barh(top_tokens[::-1], top_coefficients[::-1], color="lightblue")
    plt.xlabel("Aggregate Coefficient Value", fontsize=12)
    plt.ylabel("Tokens", fontsize=12)
    plt.title(f"Top Tokens for Predicted Category '{predicted_category}'", fontsize=14)
    plt.grid(axis="x", linestyle="--", alpha=0.7)
    plt.tight_layout()

    # Save the plot as a PNG file
    plot_filename = f"{predicted_category}_top_tokens.png"
    plt.savefig(os.path.join(output_folder, plot_filename))

    # Close the plot after saving to free up memory
    plt.close()

Plotting for predicted category: portfolio
Plotting for predicted category: audited financial statements
Plotting for predicted category: capital committed invested
Plotting for predicted category: monthly quarterly annual update
Plotting for predicted category: legal additional subscription form
Plotting for predicted category: legal lpa drafts supplements redlines
Plotting for predicted category: presentation
Plotting for predicted category: risk report
Plotting for predicted category: legal partnership company agreementlpa
Plotting for predicted category: employee information
Plotting for predicted category: risk management policy
Plotting for predicted category: compliance business continuity manual
Plotting for predicted category: due diligence
Plotting for predicted category: investor letter
Plotting for predicted category: conference meeting notes
Plotting for predicted category: performance attribution
Plotting for predicted category: onepager
Plotting for predicted category: c

In [22]:
import pickle

# Assuming `model` is your trained model
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)